In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
from geopy.geocoders import Nominatim

## 1. Loading data + manipulation

In [3]:
# Define the path to the raw data file
raw_datacenters_path = Path.cwd().parent / 'data' / 'raw' / 'datacenter_map_scraped.csv'

# Load the raw data into a DataFrame
raw_datacenters = pd.read_csv(raw_datacenters_path)

# Drop the unnamed column
raw_datacenters = raw_datacenters.drop(columns='Unnamed: 0')

# Extract the provider name from the 'name' column using regex
provider_regex = r'(.*):'
raw_datacenters['provider'] = raw_datacenters['name'].str.extract(provider_regex)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/leonardogomes/Library/CloudStorage/GoogleDrive-leodavinci550@gmail.com/My Drive/Elementum/GCB/Data Centers/dc_map_project/data/raw/datacenter_map_scraped_brasil.csv'

## 2. Defining the unscrambling functions

In [138]:
def parse_field_value(value):
    """
    Parse a string in the format "{number} {category}" into a tuple (number, category).
    
    Args:
        value (str): The string to parse, e.g., "10 Servers" or "5.5 MW".
    
    Returns:
        tuple: A tuple containing (number, category). If the value is NaN or doesn't match the format,
               returns (None, None).
    """
    if pd.isna(value):
        return None, None
    
    # Remove leading/trailing whitespace
    value = value.strip()
    
    # Use regex to extract the number and category
    match = re.match(r'^\s*([\d.]+)\s+(.+)$', value)
    if match:
        number_str, category = match.groups()
        # Convert the number to float or int based on its format
        try:
            number = float(number_str) if '.' in number_str else int(number_str)
        except ValueError:
            number = number_str  # Fallback to string if conversion fails
        return number, category.strip()
    return None, None

def extract_fields_from_row(row):
    """
    Extract structured data from field columns in a row and return a Series of key-value pairs.
    
    Args:
        row (pd.Series): A row from the DataFrame.
    
    Returns:
        pd.Series: A Series where keys are categories and values are numbers.
    """
    extracted_fields = {}
    for col in field_columns:
        value = row[col]
        number, category = parse_field_value(value)
        if category is not None:
            # Keep the first occurrence of each category
            if category not in extracted_fields:
                extracted_fields[category] = number
    return pd.Series(extracted_fields)

## 2. Applying the unscrambling functions

In [139]:
# Identify columns that start with 'field' (these contain structured data)
field_columns = [col for col in raw_datacenters.columns if col.startswith('field')]

# Apply the extraction function to each row
extracted_fields = raw_datacenters.apply(extract_fields_from_row, axis=1)

# Merge the extracted fields back into the original DataFrame
clean_datacenters = pd.concat(
    [raw_datacenters.drop(columns=field_columns), extracted_fields],
    axis=1
)

## 3. Collapsing MW columns into a single one + adjusting column names

In [140]:
# First, convert both columns to numeric, coercing any non‐numeric values to NaN.
clean_datacenters["MW total power"] = pd.to_numeric(clean_datacenters["MW total power"], errors="coerce")
clean_datacenters["MW MW total power"] = pd.to_numeric(clean_datacenters["MW MW total power"], errors="coerce")

# Use combine_first to fill NaN values in "MW total power" with those from "MW MW total power"
clean_datacenters["MW total power"] = clean_datacenters["MW total power"].combine_first(clean_datacenters["MW MW total power"])

# Optionally, drop the duplicate column
clean_datacenters.drop(columns=["MW MW total power"], inplace=True)

# Dropping colocation products column, which contains terrible data
clean_datacenters.drop(columns=["colocation products"], inplace=True)

# Standardize column names by replacing spaces with underscores
clean_datacenters.columns = [colname.replace(' ', '_') for colname in clean_datacenters.columns]

In [92]:
sample = clean_datacenters.head()

In [142]:
def extract_address_components(address):
    """
    Given an address string in a format like:
      "street number street name, municipality, state [ZIP], USA"
    this function removes the trailing USA (or United States) part and returns a Series with:
      - number: The street number.
      - street: The remainder of the street address.
      - municipality: The municipality (city) name.
      - state: The state (with any ZIP code digits removed).
    If extraction fails, returns NaN for each component.
    """
    # Split the address by commas and trim whitespace
    parts = [part.strip() for part in address.split(',')]
    
    # Remove any trailing parts that are "USA" or "United States" (case-insensitive)
    while parts and parts[-1].upper() in ["USA", "UNITED STATES"]:
        parts.pop()
    
    # Expect at least three parts now: [street, municipality, state]
    # Extract the street portion (first part) and split it into number and street name.
    if parts and len(parts[0]) > 0:
        m = re.match(r'^(\d+)\s+(.*)$', parts[0])
        if m:
            number = m.group(1)
            street = m.group(2)
        else:
            # If the pattern doesn't match, return the whole thing as street.
            number, street = np.nan, parts[0]
    else:
        number, street = np.nan, np.nan

    # Municipality: if available, assume it's the next part.
    municipality = parts[1] if len(parts) >= 2 else np.nan

    # State: if available, assume it's the third part.
    # Also, remove any digits (e.g. ZIP codes) from it.
    state_candidate = parts[2] if len(parts) >= 3 else np.nan
    if isinstance(state_candidate, str):
        state = re.sub(r'\d+', '', state_candidate).strip()
    else:
        state = np.nan

    return pd.Series([number, street, municipality, state], index=['number', 'street', 'municipality', 'state'])

# Example usage: (assuming your cleaned DataFrame is named clean_datacenters)
clean_datacenters[['number', 'street', 'municipality', 'state']] = clean_datacenters['address'].apply(extract_address_components)



In [170]:
# Initialize the Nominatim geolocator with a longer timeout.
geolocator = Nominatim(user_agent="datacenter_geocoder", timeout=20)

def geocode_row(row):
    """
    Construct an address query from available components and return
    a (latitude, longitude) tuple by querying Nominatim.
    
    The row is assumed to have the following columns (possibly with NaN):
        - number: the street number (e.g., '101')
        - street: the street name (e.g., 'Herbert Drive')
        - municipality: the city (e.g., 'Boydton')
        - state: the state code or name (e.g., 'VA')
    
    This function builds a query string using the available pieces and
    appends 'USA' at the end.
    
    If the geocoding is successful, returns (latitude, longitude).
    Otherwise, returns (np.nan, np.nan).
    """
    components = []
    
    # If both number and street are available, combine them; otherwise, use what is available.
    if pd.notnull(row['number']) and pd.notnull(row['street']):
        components.append(f"{row['number']} {row['street']}")
    elif pd.notnull(row['street']):
        components.append(row['street'])
    
    # Add municipality if available.
    if pd.notnull(row['municipality']):
        components.append(row['municipality'])
    
    # Add state if available.
    if pd.notnull(row['state']):
        components.append(row['state'])
    
    # Append the country (helps Nominatim narrow down the query)
    components.append("USA")
    
    # Build the query address string.
    query_address = ", ".join(components)
    
    try:
        location = geolocator.geocode(query_address)
        # Pause briefly to respect Nominatim's rate limits.
        time.sleep(1)
        if location:
            return (location.latitude, location.longitude)
        else:
            # If no location found, return NaNs.
            return (np.nan, np.nan)
    except Exception as e:
        print(f"Error geocoding '{query_address}':", e)
        return (np.nan, np.nan)


In [171]:
# Now apply the geocoding function row by row and expand the returned tuple into new columns:
clean_datacenters[['latitude', 'longitude']] = clean_datacenters.apply(geocode_row, axis=1, result_type='expand')

Error geocoding '2000 East Lamar Boulevard, Arlington, TX, USA': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=2000+East+Lamar+Boulevard%2C+Arlington%2C+TX%2C+USA&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x169df0850>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Error geocoding '624 South Grand Avenue, Los Angeles, CA, USA': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=624+South+Grand+Avenue%2C+Los+Angeles%2C+CA%2C+USA&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x169e7bfd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Error geocoding '21715 Filigree Court, Ashburn, VA, USA': HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries e

In [185]:
successfull1 = clean_datacenters[~clean_datacenters.latitude.isna()]

In [186]:
successfull_path = Path.cwd().parent / 'data' / 'processed' / 'successful1.csv'
successfull1.to_csv(successfull_path)

In [187]:
fail = clean_datacenters[clean_datacenters.latitude.isna()]

In [188]:
fail

,url,name,address,provider,MW_total_power,miles_to_nearest_airport,sqft_colocation_space,sqft_total_space,number,street,municipality,state,latitude,longitude
0,https://datacenters.com/microsoft-azure-east-u...,Microsoft Azure: East US-Virginia,"101 Herbert Drive, Boydton, VA, USA",Microsoft Azure,NaN,175.23,NaN,1100000.0,101,Herbert Drive,Boydton,VA,NaN,NaN
5,https://datacenters.com/microsoft-azure-west-u...,Microsoft Azure: West US 2-Washington,"1515 Port Industrial Way, Quincy, WA, USA",Microsoft Azure,NaN,114.34,NaN,800000.0,1515,Port Industrial Way,Quincy,WA,NaN,NaN
8,https://datacenters.com/amazon-aws-pdx4-boardm...,"Amazon AWS: PDX4 Boardman, OR","79539 Rippee Road, Boardman, OR 97818, USA",Amazon AWS,NaN,167.77,NaN,NaN,79539,Rippee Road,Boardman,OR,NaN,NaN
14,https://datacenters.com/switch-switch-core-campus,Switch Data Centers: Switch Data Centers: CORE...,"Switch LAS VEGAS - The Core Campus, 7135 South...",Switch Data Centers: Switch Data Centers,NaN,3.73,NaN,471248.0,NaN,Switch LAS VEGAS - The Core Campus,7135 South Decatur Boulevard,Las Vegas,NaN,NaN
15,https://datacenters.com/switch-switch-citadel-...,Switch Data Centers: Switch Data Centers: CITA...,"Switch TAHOE RENO - The Citadel Campus, Superl...",Switch Data Centers: Switch Data Centers,NaN,3.73,NaN,NaN,NaN,Switch TAHOE RENO - The Citadel Campus,Superloop Circle,Sparks,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,https://datacenters.com/amazon-aws-crosby-campus,Amazon AWS: Crosby Campus,"6645 Crosby Court, Dublin, OH, USA",Amazon AWS,NaN,2.76,NaN,NaN,6645,Crosby Court,Dublin,OH,NaN,NaN
2055,https://datacenters.com/tecfusions-new-kensing...,"TECfusions: New Kensington, PA Data Center","100 Technical Drive, New Kensington, PA, USA",TECfusions,200.0,5.51,NaN,NaN,100,Technical Drive,New Kensington,PA,NaN,NaN
2056,https://datacenters.com/navisite-santa-clara,NaviSite: Santa Clara Data Center,"2720 Zanker Road, San Jose, CA, USA",NaviSite,1.2,1.90,NaN,NaN,2720,Zanker Road,San Jose,CA,NaN,NaN
2057,https://datacenters.com/opti9-technologies-neb...,Opti9 Technologies: Nebraska Data Center,"1010 N 96th St ste 100, Omaha, NE 68114, USA",Opti9 Technologies,NaN,2.71,NaN,NaN,1010,N 96th St ste 100,Omaha,NE,NaN,NaN


# Cell Tower data wrangling

In [24]:
# importing the cell tower data
towers_path = Path.cwd().parent / 'data' / 'raw' / 'cell_towers.csv'
towers = pd.read_csv(towers_path)

In [25]:
# keeping only USA towers
towers = towers[towers.mcc == 310]

# keeping only relevant columns
towers = towers[['cell', 'radio', 'lon', 'lat', 'range', 'created', 'updated']]

# converting timestamps to ISO format
towers['created'] = pd.to_datetime(towers['created'], unit='s').dt.strftime('%Y-%m-%d')
towers['updated'] = pd.to_datetime(towers['updated'], unit='s').dt.strftime('%Y-%m-%d')

In [27]:
# exporting towers dataframe to csv
towers_processed_path = Path.cwd().parent / 'data' / 'processed' / 'cell_towers_processed.csv'
towers.to_csv(towers_processed_path)